<a href="https://colab.research.google.com/github/rtdatasci/Python/blob/main/LLM/Chat_with_pdf_HG_llama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pypdf
!pip install python-dotenv
!pip install -q transformers einops accelerate langchain bitsandbytes
!pip install -q sentence_transformers
!pip install -q llama-index


In [2]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms import HuggingFaceLLM

In [3]:
# create a /Data folder in /content and add pdf files here eg. sample.pdf from https://www.africau.edu/images/default/sample.pdf
documents = SimpleDirectoryReader("/content/Data/").load_data()

In [4]:
documents

[Document(id_='1bfc156b-b54c-4bf7-ba98-97ea7fe38106', embedding=None, metadata={'page_label': '1', 'file_name': 'sample.pdf', 'file_path': '/content/Data/sample.pdf', 'creation_date': '2023-11-01', 'last_modified_date': '2023-11-01', 'last_accessed_date': '2023-11-01'}, excluded_embed_metadata_keys=['creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, hash='f5f375aa21020ac83b96fb9c762e1f1036078dd43434308e82858b23882c70de', text=' A Simple PDF File \n This is a small demonstration .pdf file - \n just for use in the Virtual Mechanics tutorials. More text. And more \n text. And more text. And more text. And more text. \n And more text. And more text. And more text. And more text. And more \n text. And more text. Boring, zzzzz. And more text. And more text. And \n more text. And more text. And more text. And more text. And more text. \n And more text. And more text. \n And m

In [5]:

from llama_index.prompts.prompts import SimpleInputPrompt


system_prompt = "You are a Q&A assistant. Your goal is to answer questions as accurately as possible based on the instructions and context provided."



# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [6]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [7]:
# access granted to xxxxxx gmail account for meta-llama
import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LangchainEmbedding, ServiceContext

embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
)


In [9]:
service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [10]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [13]:
query_engine = index.as_query_engine()
response = query_engine.query("What is the most common word in this document?")
print(response)

The most common word in the provided document is "text". It appears 21 times in the document, which is significantly more than any other word.


In [14]:
# For a conitnuous chat:
while True:
  query=input()
  response = query_engine.query(query)
  print(response)

What is the most common word in this document?
The most common word in the provided document is "text". It appears 21 times in the document, which is significantly more than any other word.
Can you summarize the document in one line?
Sure, the document appears to be a humorous mockumentary of a boring meeting, with the author using repetitive text to exaggerate the tedium of the event.
How many words are in the document?
The document contains 105 words.
How many pages are in the document?
The document has 2 pages.
Who is the author of the content in the document?
The author of the content in the document is not explicitly stated in the provided context. However, based on the content of the document, it appears to be a humorous and satirical piece, possibly written by a group or individual with a sense of humor. Without further context or information, it is difficult to determine the author's identity with certainty.
Can you give the document a short title?
Sure! Based on the provided c

KeyboardInterrupt: ignored